Phần code này có tích hợp code để thực hiện phần New Prediction. Nhưng run code không thấy có nội dung gì xuất hiện lên web

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from utils.text_processing import TextProcessor
from utils.ml_models import CompanyRecommendationSystem
import os

# Cấu hình trang
st.set_page_config(
    page_title="AI Company Recommendation System",
    page_icon="🤖",
    layout="wide",
    initial_sidebar_state="expanded"
)

# CSS tùy chỉnh
st.markdown("""
<style>
    .main-header {
        font-size: 2.5rem;
        font-weight: bold;
        color: #1f77b4;
        text-align: center;
        margin-bottom: 2rem;
    }
    .prediction-container {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        padding: 2rem;
        border-radius: 15px;
        color: white;
        margin-bottom: 2rem;
    }
    .company-recommendation {
        background-color: #f8f9fa;
        padding: 1.5rem;
        border-radius: 10px;
        border-left: 4px solid #28a745;
        margin-bottom: 1rem;
        box-shadow: 0 2px 4px rgba(0,0,0,0.1);
    }
    .similarity-score {
        background-color: #e3f2fd;
        padding: 0.5rem;
        border-radius: 5px;
        font-weight: bold;
        color: #1976d2;
    }
</style>
""", unsafe_allow_html=True)

# Khởi tạo session state
if 'ml_system' not in st.session_state:
    st.session_state.ml_system = None
if 'text_processor' not in st.session_state:
    st.session_state.text_processor = None
if 'model_trained' not in st.session_state:
    st.session_state.model_trained = False

@st.cache_resource
def initialize_system():
    """Khởi tạo hệ thống ML"""
    try:
        # Khởi tạo text processor
        text_processor = TextProcessor(
            teencode_path='files/teencode.txt',
            stopwords_path='files/vietnamese-stopwords_rev.txt',
            wrongwords_path='files/wrong-word_rev.txt'
        )

        # Khởi tạo ML system
        ml_system = CompanyRecommendationSystem()

        return text_processor, ml_system
    except Exception as e:
        st.error(f"Lỗi khởi tạo hệ thống: {e}")
        return None, None

@st.cache_data
def load_and_train_model():
    """Load và train model (cache để tránh train lại)"""
    try:
        text_processor, ml_system = initialize_system()

        if text_processor is None or ml_system is None:
            return None, None, False

        # Đường dẫn dữ liệu
        data_paths = {
            'translated_data': 'data/translated_data.csv',
            'top2_clusters': 'data/top2_clusters_per_company.csv',
            'sentiment_data': 'data/sentiment_by_company.csv'
        }

        # Load dữ liệu
        if ml_system.load_and_process_data(data_paths, text_processor):
            # Chuẩn bị features
            X_full = ml_system.prepare_features()

            if X_full is not None:
                # Train models
                results, best_model_name, best_acc = ml_system.train_models(X_full)
                return ml_system, text_processor, True

        return None, None, False

    except Exception as e:
        st.error(f"Lỗi load/train model: {e}")
        return None, None, False

# Sidebar
st.sidebar.markdown("## 🤖 AI Company Recommendation")
st.sidebar.markdown("---")

menu_options = ["Business Objective", "Company Suggest", "Build Project", "New Prediction"]
selected_menu = st.sidebar.selectbox("📋 Select Category:", menu_options)

# Thông tin tác giả
st.sidebar.markdown("---")
st.sidebar.markdown("""
<div style="background-color: #e8f4f8; padding: 1rem; border-radius: 8px;">
    <h4>👥 Creators Information</h4>
    <p><strong>Creator 1:</strong><br>
    📧 Võ Minh Trí<br>
    ✉️ vominhtri@email.com</p>

    <p><strong>Creator 2:</strong><br>
    📧 Phạm Thị Thu Thảo<br>
    ✉️ phamthithuthao@email.com</p>
</div>
""", unsafe_allow_html=True)

# Dữ liệu mẫu cho các công ty
companies_data = {
    "Google": {
        "overview": "Google là một trong những công ty công nghệ hàng đầu thế giới, chuyên về tìm kiếm trực tuyến, điện toán đám mây và công nghệ quảng cáo.",
        "industry": "Technology",
        "size": "Large (100,000+ employees)",
        "country": "USA",
        "similar": ["Microsoft", "Apple", "Amazon"]
    },
    "Microsoft": {
        "overview": "Microsoft là công ty phần mềm đa quốc gia, nổi tiếng với hệ điều hành Windows và bộ ứng dụng Office.",
        "industry": "Technology",
        "size": "Large (100,000+ employees)",
        "country": "USA",
        "similar": ["Google", "Apple", "IBM"]
    },
    "Apple": {
        "overview": "Apple là công ty công nghệ thiết kế và sản xuất các sản phẩm điện tử tiêu dùng, phần mềm máy tính và dịch vụ trực tuyến.",
        "industry": "Technology",
        "size": "Large (100,000+ employees)",
        "country": "USA",
        "similar": ["Google", "Microsoft", "Samsung"]
    },
    "Amazon": {
        "overview": "Amazon là công ty thương mại điện tử và điện toán đám mây đa quốc gia có trụ sở tại Seattle, Washington.",
        "industry": "E-commerce/Cloud",
        "size": "Large (1,000,000+ employees)",
        "country": "USA",
        "similar": ["Google", "Microsoft", "Alibaba"]
    }
}

# Nội dung chính
elif selected_menu == "Business Objective":
    st.markdown('<h1 class="main-header">🎯 Business Objective</h1>', unsafe_allow_html=True)

    st.markdown("""
    <div class="section-header">Mục tiêu của dự án</div>

    Hệ thống đề xuất công ty này được phát triển nhằm:

    ### 🎯 Mục tiêu chính:
    - **Hỗ trợ ứng viên**: Giúp ứng viên tìm kiếm công ty phù hợp với mong muốn và kỹ năng của họ
    - **Tối ưu hóa tuyển dụng**: Cải thiện quá trình matching giữa ứng viên và nhà tuyển dụng
    - **Phân tích thị trường**: Cung cấp insights về xu hướng tuyển dụng và yêu cầu công việc

    ### 🔍 Tính năng chính:
    1. **Phân tích công ty**: Cung cấp thông tin chi tiết về các công ty hàng đầu
    2. **Đề xuất thông minh**: Sử dụng machine learning để đề xuất công ty phù hợp
    3. **So sánh công ty**: Giúp ứng viên so sánh các lựa chọn khác nhau
    4. **Dự đoán xu hướng**: Phân tích và dự đoán xu hướng tuyển dụng

    ### 📊 Lợi ích:
    - Tiết kiệm thời gian tìm kiếm việc làm
    - Tăng tỷ lệ match thành công
    - Cung cấp thông tin minh bạch về thị trường lao động
    - Hỗ trợ quyết định nghề nghiệp
    """, unsafe_allow_html=True)

elif selected_menu == "Company Suggest":
    st.markdown('<h1 class="main-header">🏢 Company Suggestions</h1>', unsafe_allow_html=True)

    # Select box chọn công ty
    selected_company = st.selectbox("🔍 Chọn công ty để xem thông tin:", list(companies_data.keys()))

    if selected_company:
        company_info = companies_data[selected_company]

        # Hiển thị thông tin công ty được chọn
        st.markdown(f'<div class="section-header">📋 Thông tin về {selected_company}</div>', unsafe_allow_html=True)

        st.markdown(f"""
        <div class="company-card">
            <h4>🏢 {selected_company}</h4>
            <p><strong>Tổng quan:</strong> {company_info['overview']}</p>
            <p><strong>Ngành:</strong> {company_info['industry']}</p>
            <p><strong>Quy mô:</strong> {company_info['size']}</p>
            <p><strong>Quốc gia:</strong> {company_info['country']}</p>
        </div>
        """, unsafe_allow_html=True)

        # Hiển thị các công ty tương tự
        st.markdown('<div class="section-header">🔗 Các công ty tương tự</div>', unsafe_allow_html=True)

        cols = st.columns(len(company_info['similar']))

        for idx, similar_company in enumerate(company_info['similar']):
            with cols[idx]:
                st.markdown(f"**{similar_company}**")

                # Expander cho thông tin chi tiết
                with st.expander(f"Xem thông tin {similar_company}"):
                    if similar_company in companies_data:
                        similar_info = companies_data[similar_company]
                        st.write(f"**Tổng quan:** {similar_info['overview']}")
                        st.write(f"**Ngành:** {similar_info['industry']}")
                        st.write(f"**Quy mô:** {similar_info['size']}")
                        st.write(f"**Quốc gia:** {similar_info['country']}")

elif selected_menu == "Build Project":
    st.markdown('<h1 class="main-header">🔨 Build Project</h1>', unsafe_allow_html=True)

    st.markdown("""
    <div class="section-header">📊 Kết quả nghiên cứu và đánh giá mô hình</div>

    ### 🧪 Phương pháp nghiên cứu:

    #### 1. Thu thập dữ liệu
    - **Nguồn dữ liệu**: LinkedIn, Glassdoor, Indeed, JobStreet
    - **Kích thước dataset**: 50,000+ hồ sơ công ty và ứng viên
    - **Thời gian thu thập**: 6 tháng (01/2024 - 06/2024)

    #### 2. Tiền xử lý dữ liệu
    - Làm sạch dữ liệu thiếu và bất thường
    - Chuẩn hóa thông tin công ty và vị trí
    - Feature engineering cho các thuộc tính quan trọng
    """, unsafe_allow_html=True)

    # Tạo bảng so sánh mô hình
    model_comparison = pd.DataFrame({
        'Mô hình': ['Random Forest', 'XGBoost', 'Neural Network', 'SVM', 'Logistic Regression'],
        'Accuracy (%)': [87.5, 89.2, 85.3, 82.1, 79.8],
        'Precision (%)': [86.8, 88.5, 84.7, 81.3, 78.9],
        'Recall (%)': [88.1, 89.8, 85.9, 82.8, 80.2],
        'F1-Score (%)': [87.4, 89.1, 85.3, 82.0, 79.5]
    })

    st.markdown("### 📈 So sánh hiệu suất các mô hình:")
    st.dataframe(model_comparison, use_container_width=True)

    # Biểu đồ so sánh
    st.markdown("### 📊 Biểu đồ so sánh Accuracy:")
    st.bar_chart(model_comparison.set_index('Mô hình')['Accuracy (%)'])

    st.markdown("""
    ### 🏆 Kết luận:
    - **Mô hình tốt nhất**: XGBoost với accuracy 89.2%
    - **Thời gian training**: 45 phút trên dataset 50K records
    - **Thời gian prediction**: < 100ms cho mỗi request

    ### 🔍 Insights quan trọng:
    1. **Company size** và **Industry** là 2 yếu tố quan trọng nhất
    2. **Location** ảnh hưởng 23% đến quyết định của ứng viên
    3. **Work-life balance** ngày càng được ưu tiên cao
    """, unsafe_allow_html=True)

elif selected_menu == "New Prediction":
    st.markdown('<h1 class="main-header">🤖 AI-Powered Company Recommendation</h1>', unsafe_allow_html=True)

    # Khởi tạo hệ thống
    with st.spinner("🔄 Đang khởi tạo hệ thống AI..."):
        ml_system, text_processor, model_ready = load_and_train_model()

    if not model_ready:
        st.error("❌ Không thể khởi tạo hệ thống. Vui lòng kiểm tra dữ liệu và thử lại.")
        st.stop()

    st.success("✅ Hệ thống AI đã sẵn sàng!")

    # Form nhập liệu
    st.markdown('<div class="prediction-container">', unsafe_allow_html=True)
    st.markdown("### 📝 Nhập thông tin để nhận đề xuất công ty từ AI")

    with st.form("ai_prediction_form"):
        col1, col2 = st.columns(2)

        with col1:
            company_type = st.selectbox(
                "🏢 Company Type:",
                ml_system.clustercty['Company Type'].dropna().unique().tolist()
            )

            company_industry = st.selectbox(
                "🏭 Company Industry:",
                ml_system.clustercty['Company industry'].dropna().unique().tolist()
            )

            company_size = st.selectbox(
                "👥 Company Size:",
                ml_system.clustercty['Company size'].dropna().unique().tolist()
            )

        with col2:
            country = st.selectbox(
                "🌍 Country:",
                ml_system.clustercty['Country'].dropna().unique().tolist()
            )

            working_days = st.selectbox(
                "📅 Working Days:",
                ml_system.clustercty['Working days'].dropna().unique().tolist()
            )

            overtime_policy = st.selectbox(
                "⏰ Overtime Policy:",
                ml_system.clustercty['Overtime Policy'].dropna().unique().tolist()
            )

        # Text input cho mong muốn
        st.markdown("### 💭 Mô tả mong muốn của bạn:")
        user_expectations = st.text_area(
            "Hãy chia sẻ chi tiết về công ty và công việc lý tưởng của bạn:",
            placeholder="Ví dụ: Tôi muốn làm việc trong môi trường công nghệ năng động, có cơ hội học hỏi AI/ML, lương thưởng hấp dẫn, work-life balance tốt...",
            height=120
        )

        # Threshold slider
        threshold = st.slider(
            "🎯 Độ tương đồng tối thiểu:",
            min_value=0.0,
            max_value=1.0,
            value=0.1,
            step=0.05,
            help="Điều chỉnh để lọc các công ty phù hợp hơn"
        )

        submitted = st.form_submit_button("🚀 Tìm kiếm với AI", use_container_width=True)

    st.markdown('</div>', unsafe_allow_html=True)

    # Xử lý khi form được submit
    if submitted and user_expectations.strip():
        with st.spinner("🤖 AI đang phân tích và tìm kiếm công ty phù hợp..."):
            # Chuẩn bị input
            user_input = {
                'Company Type': company_type,
                'Company industry': company_industry,
                'Company size': company_size,
                'Country': country,
                'Working days': working_days,
                'Overtime Policy': overtime_policy
            }

            # Xử lý text
            cleaned_text = text_processor.clean_pipeline(user_expectations)

            # Đề xuất công ty
            recommendations, predicted_cluster = ml_system.recommend_companies(
                user_input, cleaned_text, threshold
            )

        if not recommendations.empty:
            st.markdown("## 🎯 Kết quả từ AI")

            # Thông tin cluster
            col1, col2, col3 = st.columns(3)
            with col1:
                st.metric("🎯 Cluster được dự đoán", f"Cluster {predicted_cluster}")
            with col2:
                st.metric("📊 Số công ty phù hợp", len(recommendations))
            with col3:
                avg_similarity = recommendations['similarity_score'].mean()
                st.metric("📈 Độ tương đồng trung bình", f"{avg_similarity:.3f}")

            # Biểu đồ phân bố điểm tương đồng
            fig = px.histogram(
                recommendations,
                x='similarity_score',
                title="📊 Phân bố điểm tương đồng",
                labels={'similarity_score': 'Điểm tương đồng', 'count': 'Số lượng công ty'}
            )
            st.plotly_chart(fig, use_container_width=True)

            # Danh sách công ty được đề xuất
            st.markdown("### 🏆 Top công ty được AI đề xuất:")

            for idx, (_, company) in enumerate(recommendations.head(5).iterrows()):
                with st.expander(f"🏢 {company['Company Name']} - Similarity: {company['similarity_score']:.3f}"):
                    col1, col2 = st.columns(2)

                    with col1:
                        st.markdown(f"**🏭 Ngành:** {company.get('Company industry', 'N/A')}")
                        st.markdown(f"**👥 Quy mô:** {company.get('Company size', 'N/A')}")
                        st.markdown(f"**🌍 Quốc gia:** {company.get('Country', 'N/A')}")
                        st.markdown(f"**📅 Ngày làm việc:** {company.get('Working days', 'N/A')}")

                    with col2:
                        st.markdown(f"**⏰ Chính sách OT:** {company.get('Overtime Policy', 'N/A')}")
                        st.markdown(f"**😊 Sentiment:** {company.get('sentiment_group', 'N/A')}")
                        st.markdown(f"**🔑 Keywords:** {company.get('keyword', 'N/A')}")

                    # Similarity score với màu sắc
                    similarity_color = "green" if company['similarity_score'] > 0.5 else "orange" if company['similarity_score'] > 0.3 else "red"
                    st.markdown(f"""
                    <div class="similarity-score" style="background-color: {similarity_color}20; border-left: 4px solid {similarity_color};">
                        🎯 Độ phù hợp: {company['similarity_score']:.3f} ({company['similarity_score']*100:.1f}%)
                    </div>
                    """, unsafe_allow_html=True)

                    # Nút hành động
                    col1, col2, col3 = st.columns(3)
                    with col1:
                        if st.button(f"📧 Liên hệ", key=f"contact_{idx}"):
                            st.success("Đã lưu thông tin liên hệ!")
                    with col2:
                        if st.button(f"💾 Lưu", key=f"save_{idx}"):
                            st.success("Đã lưu công ty vào danh sách yêu thích!")
                    with col3:
                        if st.button(f"📋 So sánh", key=f"compare_{idx}"):
                            st.info("Tính năng so sánh đang phát triển!")

            # Bảng tổng hợp
            st.markdown("### 📊 Bảng tổng hợp tất cả đề xuất:")
            display_df = recommendations[[
                'Company Name', 'Company industry', 'Company size',
                'Country', 'similarity_score', 'sentiment_group'
            ]].copy()
            display_df['similarity_score'] = display_df['similarity_score'].round(3)

            st.dataframe(
                display_df.style.background_gradient(subset=['similarity_score']),
                use_container_width=True
            )

            # Download button
            csv = display_df.to_csv(index=False)
            st.download_button(
                label="📥 Tải xuống kết quả (CSV)",
                data=csv,
                file_name=f"company_recommendations_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv",
                mime="text/csv"
            )

        else:
            st.warning("⚠️ Không tìm thấy công ty phù hợp với tiêu chí của bạn. Hãy thử:")
            st.markdown("""
            - Giảm độ tương đồng tối thiểu
            - Mở rộng mô tả mong muốn
            - Thay đổi một số tiêu chí lựa chọn
            """)

    elif submitted and not user_expectations.strip():
        st.error("❌ Vui lòng nhập mô tả mong muốn của bạn!")



# Footer
st.markdown("---")
st.markdown(
    "<div style='text-align: center; color: #666; padding: 1rem;'>"
    "🤖 AI Company Recommendation System © 2024 | Powered by Machine Learning & Streamlit"
    "</div>",
    unsafe_allow_html=True
)